# Практика
Используемые библиотеки

In [126]:
import pandas as pd
import numpy as np
import re # for data processing
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
#headers={
#    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
#}
#def get_content(url):
#    with requests.Session() as req:
#        req.headers.update(headers)
#        r = req.get(url).content
#    return r
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = get_content(url)
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))


# Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [68]:
# Ваше решение
def is_float(str):
    '''Аналог isdigit() для float значений'''
    try:
        float(str.replace(',','.'))
        return True
    except ValueError:
        return False

In [111]:
# для цветности "б/цвета" означает менее 20 градусов платино-кобальтовой шкалы, 
# таким образом если в результатах у нас будет "б/цвета", то присвоим ему значение по максимуму - 20
def summary_maker(feature, unit, norm, result):
    '''Функция, формирующая вывод на основании названия параметра, значения результата и значений норматива'''
    norm_list = re.split(r' |-',norm)
    res = float(str(result).replace('б/цвета', '20'))
    maximum = max([float(i.replace(',','.')) for i in norm_list if is_float(i)])
    minimum = min([float(i.replace(',','.')) for i in norm_list if is_float(i)])
    unit_dict = {'единицы pH':'единиц pH','баллы':'баллов','градусы':'градусов','мг-эквл/дм3':'мг-эквл/дм3','мг/дм3':'мг/дм3'}
    unit = unit_dict[unit]
    if 'не' in norm_list and 'более' in norm_list:
        if res <= minimum:
            text = f'''По показателю {feature} получен результат {result} {unit} при нормативе \
{norm} {unit}. Т.о. по данному показателю результат анализа в норме.'''
        else:
            text = f'''По показателю {feature} получен результат {result} {unit} при нормативе \
{norm} {unit}. Т.о. по данному показателю результат анализа вне рамок нормативных \
показателей.'''
    elif 'пределах' in norm_list:
        if res <= maximum and res >= minimum:
            text = f'''По показателю {feature} получен результат {result} {unit} при нормативе \
от {minimum} до {maximum} {unit}. Т.о. по данному показателю результат анализа в норме.'''
        else:
            text = f'''По показателю {feature} получен результат {result} {unit} при нормативе \
от {minimum} до {maximum} {unit}. Т.о. по данному показателю результат анализа вне \
рамок нормативных показателей.'''
    else:
        text = 'Невозможно сформировать вывод'
    return text

In [116]:
df['Вывод'] = df.apply(lambda x: summary_maker(feature=x['Показатель'], unit=x['Единица измерений'],norm=x['Норматив'], result=x['Результат анализа']), axis=1)
df=df.set_index('Показатель')
display(df)

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,По показателю pH получен результат 8.4 единиц ...
Запах,баллы,1,не более 2-3,По показателю Запах получен результат 1 баллов...
Цветность,градусы,б/цвета,не более 30,По показателю Цветность получен результат б/цв...
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,По показателю Жёсткость получен результат 9.19...
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",По показателю Аммиак и аммоний-ион (по азоту) ...
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",По показателю Нитриты (по NO2) получен результ...
Нитраты (по NO3),мг/дм3,24,не более 45,По показателю Нитраты (по NO3) получен результ...
Фосфаты (P),мг/дм3,0.36,"не более 3,5",По показателю Фосфаты (P) получен результат 0....
Хлориды (Cl),мг/дм3,200,не более 350,По показателю Хлориды (Cl) получен результат 2...


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [118]:
# Ваше решение
# o - orange (апельсин), a - apple (яблоко)
box = 5*['o']+4*['a']
box

['o', 'o', 'o', 'o', 'o', 'a', 'a', 'a', 'a']

In [152]:
# Посчитаем через np.random.choice и цикл for. Обязательно ставим replace=False, т.к. мы не можем дважды 
# вытащить один и тот же фрукт. Расчет на 1 млн повторений занял 9,1 секунд. 
# Результат 0,118833, что практически совпадает с аналитическим ответом 0,119 
n = 1000000
count = 0
choice = []
for i in range(n):
    choice = np.random.choice(box,3,replace=False)
    if list(choice) == ['o','o','o']:
        count += 1
print(count/n)

0.118833


In [151]:
# Посчитаем через random.sample  и цикл for. Расчет на 1 млн повторений занял 1,6 секунд.
# Результат 0,119092, что также практически совпадает с аналитическим ответом 0,119
n = 1000000
count = 0
choice = []
for i in range(n):
    choice = random.sample(box,3)
    if choice == ['o','o','o']:
        count += 1
print(count/n)

0.119092


In [163]:
# Посчитаем через np.random.choice и lambda-функцию. Расчет на 1 млн повторений занял 13,7 секунд.
# Результат 0,119085, что также практически совпадает с аналитическим ответом 0,119
n = 1000000
generator = (list(np.random.choice(box,3,replace=False)) for _ in range(n))
df = pd.DataFrame(generator, columns=['e1','e2','e3'])
df['result'] = df.apply(lambda x: 1 if x['e1']=='o' and x['e2']=='o' and x['e3']=='o' else 0, axis=1)
print(sum(df['result'])/len(df))

0.119085


In [162]:
# Посчитаем через random.sample  и lambda-функцию. Расчет на 1 млн повторений занял 6,3 секунд.
# Результат 0,119018, что также практически совпадает с аналитическим ответом 0,119
n = 1000000
generator = (random.sample(box,3) for _ in range(n))
df = pd.DataFrame(generator, columns=['e1','e2','e3'])
df['result'] = df.apply(lambda x: 1 if x['e1']=='o' and x['e2']=='o' and x['e3']=='o' else 0, axis=1)
print(sum(df['result'])/len(df))

0.119018


При использовании np.random.choice() результаты на больших выборках не отличаются от аналитического решения и от метода с использованием random.sample(), в то же время расчет идёт значительно дольше, чем random.sample().
При этом, подсчет через цикл for прошел быстрее, чем через генератор и lambda-функцию

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [164]:
# Ваше решение
# s - standart, n - non standart
details = 7*['s'] + 3*['n']
details

['s', 's', 's', 's', 's', 's', 's', 'n', 'n', 'n']

In [170]:
# для расчетов нам достаточно взять только первые 2 проверки деталей
n = 100000
generator = (random.sample(details,2) for _ in range(n))
df = pd.DataFrame(generator, columns=['e1','e2'])
df['result'] = df.apply(lambda x: 1 if x['e1']=='n' and x['e2']=='s' else 0, axis=1)
print(sum(df['result'])/len(df))

0.2319


Получилось 0,2319, что подтверждает аналитический ответ в 0,23333